In [1]:
import tensorflow as tf

In [18]:
model = tf.keras.models.load_model("mel_smote_v1_iter_1.h5")

In [19]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 112, 170, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 85, 56)        8288      
_________________________________________________________________
batch_normalization_5 (Batch (None, 56, 85, 56)        224       
_________________________________________________________________
activation_5 (Activation)    (None, 56, 85, 56)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 43, 56)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 28, 43, 56)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 22, 72)        1008

In [22]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
input_saved_model_dir = "mel_smote_v1_iter_1.model"
output_saved_model_dir = "mel22oct"
converter = trt.TrtGraphConverterV2(input_saved_model_dir=input_saved_model_dir)
converter.convert()
converter.save(output_saved_model_dir)

INFO:tensorflow:Linked TensorRT version: (7, 1, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 1, 3)
INFO:tensorflow:Could not find TRTEngineOp_0_0 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Assets written to: mel22oct/assets


In [24]:
saved_model_loaded = tf.saved_model.load(output_saved_model_dir)


In [2]:
import numpy as np

In [52]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt

input_saved_model_dir = "mel_smote_v1_iter_1.model"
output_saved_model_dir = "mel22oct_nvidia"

conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
conversion_params = conversion_params._replace(max_workspace_size_bytes=(1<<32))
conversion_params = conversion_params._replace(precision_mode="FP16")
conversion_params = conversion_params._replace(maximum_cached_engines=1)

converter = trt.TrtGraphConverterV2(input_saved_model_dir=input_saved_model_dir,
    conversion_params=conversion_params, max_batch_size =1)
converter.convert()
def my_input_fn():
    for _ in range(1):
        Inp1 = np.random.normal(size=(8,112,170,3)).astype(np.float32)
#         inp2 = np.random.normal(size=(1, 112, 170, 3)).astype(np.float32)
        yield Inp1

    
converter.build(input_fn=my_input_fn)
converter.save(output_saved_model_dir)

saved_model_loaded = tf.saved_model.load(output_saved_model_dir, tags=[tag_constants.SERVING])
graph_func = saved_model_loaded.signatures[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
frozen_func = convert_to_constants.convert_variables_to_constants_v2(graph_func)
output = frozen_func(input_data)[0].numpy()

TypeError: __init__() got an unexpected keyword argument 'max_batch_size'

In [53]:
print('Converting to TF-TRT FP32...')
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(precision_mode=trt.TrtPrecisionMode.FP32,
                                                               max_workspace_size_bytes=8000000000)

input_saved_model_dir = "mel_smote_v1_iter_1.model"
converter = trt.TrtGraphConverterV2(input_saved_model_dir=input_saved_model_dir,
                                    conversion_params=conversion_params)
converter.convert()
converter.save(output_saved_model_dir='mel_collab')
print('Done Converting to TF-TRT FP32')

Converting to TF-TRT FP32...
INFO:tensorflow:Linked TensorRT version: (7, 1, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 1, 3)
INFO:tensorflow:Could not find TRTEngineOp_16_0 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Assets written to: mel_collab/assets
Done Converting to TF-TRT FP32


In [54]:
print("Tensorflow version: ", tf.version.VERSION)

Tensorflow version:  2.3.0


In [3]:
from tensorflow.keras.preprocessing import image

In [8]:
def predict_tftrt(input_saved_model):
    """Runs prediction on a single image and shows the result.
    input_saved_model (string): Name of the input model stored in the current dir
    """
    img_path = './testIMG.png'  # Siberian_husky
    img = image.load_img(img_path, target_size=(112, 170))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    #x = preprocess_input(x)
    x = tf.constant(x)
    
    saved_model_loaded = tf.saved_model.load(input_saved_model)
    signature_keys = list(saved_model_loaded.signatures.keys())
    print(signature_keys)

    infer = saved_model_loaded.signatures['serving_default']
    print(infer.structured_outputs)

    labeling = infer(x)
    preds = labeling['probs'].numpy()
    print('{} - Predicted: {}'.format(img_path, decode_predictions(preds, top=3)[0]))
    plt.subplot(2,2,1)
    plt.imshow(img);
    plt.axis('off');
    #plt.title(decode_predictions(preds, top=3)[0][0][1])

In [9]:
predict_tftrt('mel_collab')

['serving_default']
{'output': TensorSpec(shape=<unknown>, dtype=tf.float32, name='output')}


KeyError: 'probs'

This is the final evaluation that worked

In [11]:
input_saved_model = 'mel_collab'

In [21]:
"""Runs prediction on a single image and shows the result.
input_saved_model (string): Name of the input model stored in the current dir
"""
img_path = './ROUA_sam.png'  # Siberian_husky
img = image.load_img(img_path, target_size=(112, 170))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
#x = preprocess_input(x)
x = tf.constant(x)

saved_model_loaded = tf.saved_model.load(input_saved_model)
signature_keys = list(saved_model_loaded.signatures.keys())
print(signature_keys)

infer = saved_model_loaded.signatures['serving_default']
print(infer.structured_outputs)



['serving_default']
{'output': TensorSpec(shape=<unknown>, dtype=tf.float32, name='output')}


In [22]:
labeling = infer(x)

In [23]:
labeling.keys()

dict_keys(['output'])

In [24]:

preds = labeling['output'].numpy()
print(preds)
#print('{} - Predicted: {}'.format(img_path, decode_predictions(preds, top=3)[0]))
plt.subplot(2,2,1)
plt.imshow(img);
plt.axis('off');
#plt.title(decode_predictions(preds, top=3)[0][0][1])

[[0. 0. 0. 0. 1. 0. 0. 0.]]


NameError: name 'plt' is not defined